In [1]:
from kedro_datasets.pandas import ParquetDataset
import pandas as pd

#kobe_raw_dev = catalog.load(name='kobe_raw_dev@parquet')
pd.options.display.max_columns = None
pd.options.display.max_rows = None
dataset = ParquetDataset(filepath='../data/01_raw/dataset_kobe_dev.parquet').load()

In [2]:
dataset = dataset.drop(columns=['action_type','combined_shot_type','game_event_id','game_id','loc_x','loc_y','season','seconds_remaining','shot_type','shot_zone_area','shot_zone_basic','shot_zone_range','team_id','team_name','game_date','matchup','opponent','shot_id'])

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24271 entries, 0 to 30696
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   lat                24271 non-null  float64
 1   lon                24271 non-null  float64
 2   minutes_remaining  24271 non-null  int64  
 3   period             24271 non-null  int64  
 4   playoffs           24271 non-null  int64  
 5   shot_distance      24271 non-null  int64  
 6   shot_made_flag     20285 non-null  float64
dtypes: float64(3), int64(4)
memory usage: 1.5 MB


In [4]:
dataset = dataset.dropna()

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20285 entries, 1 to 30696
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   lat                20285 non-null  float64
 1   lon                20285 non-null  float64
 2   minutes_remaining  20285 non-null  int64  
 3   period             20285 non-null  int64  
 4   playoffs           20285 non-null  int64  
 5   shot_distance      20285 non-null  int64  
 6   shot_made_flag     20285 non-null  float64
dtypes: float64(3), int64(4)
memory usage: 1.2 MB


In [6]:
dataset = dataset.dropna()

In [7]:
dataset.head()


,lat,lon,minutes_remaining,period,playoffs,shot_distance,shot_made_flag
1,34.0443,-118.4268,10,1,0,15,0.0
2,33.9093,-118.3708,7,1,0,16,1.0
3,33.8693,-118.1318,6,1,0,22,0.0
4,34.0443,-118.2698,6,2,0,0,1.0
5,34.0553,-118.4148,9,3,0,14,0.0


In [8]:
from pycaret.classification import *

In [9]:
experiment = ClassificationExperiment()
experiment.setup(data=dataset, target='shot_made_flag', log_experiment='mlflow', experiment_name='kobe_pycarret')#, session_id=2742)

,Description,Value
0,Session id,3926
1,Target,shot_made_flag
2,Target type,Binary
3,Original data shape,"(20285, 7)"
4,Transformed data shape,"(20285, 7)"
5,Transformed train set shape,"(14199, 7)"
6,Transformed test set shape,"(6086, 7)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


In [10]:
experiment.get_metrics()

,Name,Display Name,Score Function,Scorer,Target,Args,Greater is Better,Multiclass,Custom
ID,,,,,,,,,
acc,Accuracy,Accuracy,<function accuracy_score at 0x000002705CF500E0>,accuracy,pred,{},True,True,False
auc,AUC,AUC,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(roc_auc_score, response_method=('d...",pred_proba,"{'average': 'weighted', 'multi_class': 'ovr'}",True,True,False
recall,Recall,Recall,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(recall_score, response_method='pre...",pred,{'average': 'weighted'},True,True,False
precision,Precision,Prec.,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(precision_score, response_method='...",pred,{'average': 'weighted'},True,True,False
f1,F1,F1,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(f1_score, response_method='predict...",pred,{'average': 'weighted'},True,True,False
kappa,Kappa,Kappa,<function cohen_kappa_score at 0x000002705CF50...,"make_scorer(cohen_kappa_score, response_method...",pred,{},True,True,False
mcc,MCC,MCC,<function matthews_corrcoef at 0x000002705CF50...,"make_scorer(matthews_corrcoef, response_method...",pred,{},True,True,False


In [11]:
experiment.compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.5908,0.5973,0.3805,0.6153,0.4700,0.1661,0.1789,0.0430
gbc,Gradient Boosting Classifier,0.5902,0.6003,0.3888,0.6112,0.4750,0.1655,0.1770,0.0840
catboost,CatBoost Classifier,0.5814,0.5982,0.4613,0.5773,0.5128,0.1538,0.1569,2.1690
ridge,Ridge Classifier,0.5770,0.5988,0.4832,0.5667,0.5214,0.1468,0.1483,0.0050
lda,Linear Discriminant Analysis,0.5767,0.5988,0.4827,0.5663,0.5210,0.1462,0.1478,0.0060
lr,Logistic Regression,0.5760,0.5985,0.4818,0.5653,0.5201,0.1447,0.1462,0.3560
qda,Quadratic Discriminant Analysis,0.5755,0.5990,0.5239,0.5591,0.5408,0.1470,0.1473,0.0070
nb,Naive Bayes,0.5753,0.5958,0.5037,0.5614,0.5308,0.1449,0.1457,0.0050
lightgbm,Light Gradient Boosting Machine,0.5731,0.5901,0.4709,0.5636,0.5129,0.1384,0.1403,0.0660
rf,Random Forest Classifier,0.5611,0.5719,0.5307,0.5412,0.5358,0.1197,0.1197,0.1480


2025/04/01 18:10:24 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025/04/01 18:10:24 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025/04/01 18:10:25 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025/04/01 18:10:25 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025/04/01 18:10:25 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025/04/01 18:10:25 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when

AdaBoostClassifier(algorithm='SAMME.R', estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=3926)

In [12]:
best = _

In [13]:
experiment.evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [14]:
import pickle

with open('model.pkl', 'wb') as fout:
    pickle.dump(best, fout)